# Grids, Fields, and operations

Let's get into it.

## Grids

There are two primary kinds of grids:

In [104]:
rectilinear_grid = RectilinearGrid(size = (2, 3, 4),
                                   x = (0, 3),
                                   y = (0, 2),
                                   z = (0, 1),
                                   topology = (Periodic, Periodic, Bounded))

2×3×4 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 2×3×3 halo
├── Periodic x ∈ [0.0, 3.0) regularly spaced with Δx=1.5
├── Periodic y ∈ [0.0, 2.0) regularly spaced with Δy=0.666667
└── Bounded  z ∈ [0.0, 1.0] regularly spaced with Δz=0.25

In [105]:
latitude_longitude_grid = LatitudeLongitudeGrid(size = (60, 30, 1),
                                                longitude = (0, 60),
                                                latitude = (-30, 30),
                                                z = (-1, 0),
                                                topology = (Periodic, Bounded, Bounded))

60×30×1 LatitudeLongitudeGrid{Float64, Periodic, Bounded, Bounded} on CPU with 3×3×1 halo and with precomputed metrics
├── longitude: Periodic λ ∈ [0.0, 60.0)   regularly spaced with Δλ=1.0
├── latitude:  Bounded  φ ∈ [-30.0, 30.0] regularly spaced with Δφ=2.0
└── z:         Bounded  z ∈ [-1.0, 0.0]   regularly spaced with Δz=1.0

Another important grid is `OrthogonalSphericalShellGrid` but this is a bit of a WIP.

Let's work with the rectilinear grid for the rest of the tutorial.

In [106]:
grid = rectilinear_grid

2×3×4 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 2×3×3 halo
├── Periodic x ∈ [0.0, 3.0) regularly spaced with Δx=1.5
├── Periodic y ∈ [0.0, 2.0) regularly spaced with Δy=0.666667
└── Bounded  z ∈ [0.0, 1.0] regularly spaced with Δz=0.25

## Fields

With a grid we can start making `Field`.
Every `Field` has a location in each direction on the staggered grid.
The location of a `Field` is described by a 3-tuple; for example tracers in Oceananigans
are located at `(Center, Center, Center)`. Fields like tracers can be constructed using the shorthand `CenterField`,

In [107]:
c = CenterField(grid)

2×3×4 Field{Center, Center, Center} on RectilinearGrid on CPU
├── grid: 2×3×4 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 2×3×3 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: Periodic, east: Periodic, south: Periodic, north: Periodic, bottom: ZeroFlux, top: ZeroFlux, immersed: ZeroFlux
└── data: 6×9×10 OffsetArray(::Array{Float64, 3}, -1:4, -2:6, -2:7) with eltype Float64 with indices -1:4×-2:6×-2:7
    └── max=0.0, min=0.0, mean=0.0

As another example, in the "C-grid" discretization the velocity component in the x-direction is located at cell interfaces in $x$, such that

In [108]:
u = Field{Face, Center, Center}(grid)

2×3×4 Field{Face, Center, Center} on RectilinearGrid on CPU
├── grid: 2×3×4 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 2×3×3 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: Periodic, east: Periodic, south: Periodic, north: Periodic, bottom: ZeroFlux, top: ZeroFlux, immersed: ZeroFlux
└── data: 6×9×10 OffsetArray(::Array{Float64, 3}, -1:4, -2:6, -2:7) with eltype Float64 with indices -1:4×-2:6×-2:7
    └── max=0.0, min=0.0, mean=0.0

### Setting `Field` data

`Field`s aren't useful until you can set their values.
`Field`s can be set with constants,

In [109]:
set!(c, 1)

2×3×4 Field{Center, Center, Center} on RectilinearGrid on CPU
├── grid: 2×3×4 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 2×3×3 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: Periodic, east: Periodic, south: Periodic, north: Periodic, bottom: ZeroFlux, top: ZeroFlux, immersed: ZeroFlux
└── data: 6×9×10 OffsetArray(::Array{Float64, 3}, -1:4, -2:6, -2:7) with eltype Float64 with indices -1:4×-2:6×-2:7
    └── max=1.0, min=1.0, mean=1.0

functions,

In [110]:
random_data(x, y, z) = randn()
set!(c, random_data)

2×3×4 Field{Center, Center, Center} on RectilinearGrid on CPU
├── grid: 2×3×4 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 2×3×3 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: Periodic, east: Periodic, south: Periodic, north: Periodic, bottom: ZeroFlux, top: ZeroFlux, immersed: ZeroFlux
└── data: 6×9×10 OffsetArray(::Array{Float64, 3}, -1:4, -2:6, -2:7) with eltype Float64 with indices -1:4×-2:6×-2:7
    └── max=2.5106, min=-2.06922, mean=0.168609

and arrays,

In [111]:
random_array = rand(size(c)...)
set!(c, random_array)

2×3×4 Field{Center, Center, Center} on RectilinearGrid on CPU
├── grid: 2×3×4 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 2×3×3 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: Periodic, east: Periodic, south: Periodic, north: Periodic, bottom: ZeroFlux, top: ZeroFlux, immersed: ZeroFlux
└── data: 6×9×10 OffsetArray(::Array{Float64, 3}, -1:4, -2:6, -2:7) with eltype Float64 with indices -1:4×-2:6×-2:7
    └── max=0.997478, min=0.0767212, mean=0.396486

`Field` values can be inspected by indexing,

In [112]:
c[1, 1, 1]

0.5497478509170294

but note that this won't work on the GPU (or rather, it is very slow and it will throw an error).
The values of a `Field` are stored in `Field.data`,

In [113]:
c.data

6×9×10 OffsetArray(::Array{Float64, 3}, -1:4, -2:6, -2:7) with eltype Float64 with indices -1:4×-2:6×-2:7:
[:, :, -2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, -1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 0] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 1] =
 0.0  0.0  0.0  0.0       0.0 

Which is an `OffsetArray` that is itself wrapped around an underlying plain vanilla `Array`.
The underlying array can be accessed using the function `parent` (or by writing `c.data.parent`):

In [114]:
parent(c)

6×9×10 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 4] =
 0.0  0.0  0.0  0.0       0.0       0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0       0.0       0.0       0.0  0.

Note that a lot of the data is `0.0`. That's because `c` has "halo regions" that surround the meaningful, "interior" data. To inspect just the `interior` data of `c` we call

In [115]:
interior(c)

2×3×4 view(::Array{Float64, 3}, 3:4, 4:6, 4:7) with eltype Float64:
[:, :, 1] =
 0.549748  0.151851  0.552482
 0.401002  0.930264  0.155449

[:, :, 2] =
 0.17789   0.145093  0.203142
 0.323462  0.367217  0.0767212

[:, :, 3] =
 0.358593  0.95973   0.399396
 0.997478  0.193344  0.116034

[:, :, 4] =
 0.775553  0.100869  0.160673
 0.315062  0.541775  0.562844

### Statistics

There's a few simple things we can do,

In [116]:
using Statistics

mean(c)

0.3964863374449474

In [117]:
maximum(c)

0.997478080808811

In [118]:
minimum(c)

0.0767212375514742

### Operations

Here's where things get interesting.
Oceananigans has a system for building expression trees that represent calculus, arithmetic, and other operations on and between `Field`s.
For example,

In [119]:
two_c = 2 * c

BinaryOperation at (Center, Center, Center)
├── grid: 2×3×4 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 2×3×3 halo
└── tree: 
    * at (Center, Center, Center)
    ├── 2
    └── 2×3×4 Field{Center, Center, Center} on RectilinearGrid on CPU

`two_c` represents the operation `2 * c`. It can be indexed into, like a `Field`,

In [120]:
two_c[1, 1, 1] == 2 * c[1, 1, 1]

true

But `two_c` doesn't actually have any underlying data -- no memory has been allocated to store the result `2 * c`.
Instead, `two_c` is a _lazy_ representation of `2 * c`.
To allocate memory for `two_c` we wrap it inside a `Field`,

In [121]:
two_c_field = Field(two_c)

2×3×4 Field{Center, Center, Center} on RectilinearGrid on CPU
├── grid: 2×3×4 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 2×3×3 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: Periodic, east: Periodic, south: Periodic, north: Periodic, bottom: ZeroFlux, top: ZeroFlux, immersed: ZeroFlux
├── operand: BinaryOperation at (Center, Center, Center)
├── status: time=0.0
└── data: 6×9×10 OffsetArray(::Array{Float64, 3}, -1:4, -2:6, -2:7) with eltype Float64 with indices -1:4×-2:6×-2:7
    └── max=0.0, min=0.0, mean=0.0

Note that it's initialized to `0`.
We've got to `compute!` with intention, so

In [122]:
compute!(two_c)

BinaryOperation at (Center, Center, Center)
├── grid: 2×3×4 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 2×3×3 halo
└── tree: 
    * at (Center, Center, Center)
    ├── 2
    └── 2×3×4 Field{Center, Center, Center} on RectilinearGrid on CPU

Operations can get complicated.
For example

In [123]:
∇²c = ∂x(∂x(c)) + ∂y(∂y(c)) + ∂z(∂z(c))

MultiaryOperation at (Center, Center, Center)
├── grid: 2×3×4 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 2×3×3 halo
└── tree: 
    + at (Center, Center, Center)
    ├── ∂xᶜᶜᶜ at (Center, Center, Center) via identity
    │   └── ∂xᶠᶜᶜ at (Face, Center, Center) via identity
        │   └── 2×3×4 Field{Center, Center, Center} on RectilinearGrid on CPU
    ├── ∂yᶜᶜᶜ at (Center, Center, Center) via identity
    │   └── ∂yᶜᶠᶜ at (Center, Face, Center) via identity
        │   └── 2×3×4 Field{Center, Center, Center} on RectilinearGrid on CPU
    └── ∂zᶜᶜᶜ at (Center, Center, Center) via identity
        └── ∂zᶜᶜᶠ at (Center, Center, Face) via identity
            └── 2×3×4 Field{Center, Center, Center} on RectilinearGrid on CPU

We can even write a simple code to solve the heat equation,

$$ c^{n+1} = c^n + Δt ∇²c^n $$

In [124]:
using Oceananigans.BoundaryConditions: fill_halo_regions!

set!(c, random_data)
@show extrema(c)
@show mean(c)

Δt = 1e-3

for n = 1:1000
    fill_halo_regions!(c)
    c .+= Δt * ∇²c
end

@show mean(c)
@show extrema(c)

extrema(c) = (-2.0675597013172933, 1.1302827922585965)
mean(c) = -0.2194256450025774
mean(c) = -0.22093173265346708
extrema(c) = (-0.2434727273712095, -0.1980952021304007)


(-0.2434727273712095, -0.1980952021304007)

## Reductions

Oceananigans also has a system for building lazy representations of `Reduction`.
Important reductions include `Average` and `Interval`, though other reductions like `maximum` are also supported.
For example,

In [125]:
set!(c, random_data)
c_avg = Average(c, dims=(1, 2))

mean! over dims (1, 2) of 2×3×4 Field{Center, Center, Center} on RectilinearGrid on CPU
└── operand: 2×3×4 Field{Center, Center, Center} on RectilinearGrid on CPU
    └── grid: 2×3×4 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 2×3×3 halo

Notice that building a `Reduction` requires specifying the `dims` of the reduction; `dims=(1, 2)` indicates an average over $x, y$.
Likewise, `dims=3` averages in $z$.
Unlike operations, `Reduction`s cannot be indexed into,

In [126]:
c_avg[1, 1, 1]

LoadError: MethodError: no method matching getindex(::Reduction{typeof(mean!), Field{Center, Center, Center, Nothing, RectilinearGrid{Float64, Periodic, Periodic, Bounded, Float64, Float64, Float64, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, CPU}, Tuple{Colon, Colon, Colon}, OffsetArrays.OffsetArray{Float64, 3, Array{Float64, 3}}, Float64, FieldBoundaryConditions{BoundaryCondition{Oceananigans.BoundaryConditions.Periodic, Nothing}, BoundaryCondition{Oceananigans.BoundaryConditions.Periodic, Nothing}, BoundaryCondition{Oceananigans.BoundaryConditions.Periodic, Nothing}, BoundaryCondition{Oceananigans.BoundaryConditions.Periodic, Nothing}, BoundaryCondition{O

This is because by definition a `Reduction` cannot be computed locally;
instead we must compute the reduction in a separate step, and then evaluate it's data afterwards.
To evaluate a `Reduction` we wrap it within a `Field`,

In [127]:
c_avg_field = Field(c_avg)
compute!(c_avg_field)
interior(c_avg_field)

1×1×4 view(::Array{Float64, 3}, 1:1, 1:1, 4:7) with eltype Float64:
[:, :, 1] =
 -0.32902900538089624

[:, :, 2] =
 -0.4297366708129102

[:, :, 3] =
 0.9995448273825986

[:, :, 4] =
 -0.08421665094546449

Note that `c_avg_field` has location `(Nothing, Nothing, Center)` --- it has "no location" in $x, y$.